# Using Multimodal Chat with Palmyra X5

The Palmyra X5 model supports multimodal chat, letting you [send both](https://dev.writer.com/home/chat-with-images) text and images in the same message. You can include local images (via data URLs) or images hosted online. This enables rich, visual conversations directly in chat completions.


## Prerequisites

Before getting started, you'll need:

- A [Writer AI Studio](https://app.writer.com/register) account
- An API key, which you can obtain by following the [API Quickstart](https://dev.writer.com/api-guides/quickstart)

## Setup
Install the Writer SDK:

In [ ]:
%pip install -qU writer-sdk

Next, set the `WRITER_API_KEY` environment variable. We recommend setting it in a `.env` file in the root of your project, but this tutorial will set it in an environment variable if you don't have a `.env` file.

In [3]:
import getpass
import os
from writerai import Writer

if not os.getenv("WRITER_API_KEY"):
    os.environ["WRITER_API_KEY"] = getpass.getpass("Enter your Writer API key: ")

client = Writer()  # The Writer client automatically uses the WRITER_API_KEY environment variable to authenticate.

## Usage

The Multimodal Chat follows this format:
```json
{
    "type": "multimodal",
    "function": {
        "model": "palmyra-x5",
        "inputs": [
            {"type": "text", "text": "Your text here"},
            {"type": "image_url", "image_url": {"url": "https://example.com/image.jpg"}}
        ]
    }
}
```
The `model` field specifies which multimodal model to use (`palmyra-x5`).

The `inputs` array can contain any combination of text (type: `"text"`) and image URLs (type: `"image_url"`).

Each `image_url` can point to a hosted image or a local image converted to a base64 data URL.

This snippet lets you define the text and images you want to send in a multimodal chat. The system constructs the inputs array with this information and passes it to the LLM for a contextual response:

In [8]:
text_input = "What do you see in this image?"

image_input = "https://ichef.bbci.co.uk/news/1024/cpsprodpb/168F6/production/_130660429_1977_august_september_voyager1_1.jpg.webp"

response = client.chat.chat(
  model="palmyra-x5",
  messages=[
    {
      "role": "user",
      "content": [
        {
          "type": "text",
          "text": text_input
        },
        {
          "type": "image_url",
          "image_url": {
            "url": image_input
          }
        }
      ]
    }
  ]
)

print(response.choices[0].message.content)


The image depicts a spacecraft, specifically the Voyager 1, floating in space. 

The spacecraft is positioned on the left side of the image, with its main components visible. It has a large white dish antenna at the center, which is used for communication with Earth. Several gold-colored booms and antennae are attached to the spacecraft, likely used for scientific instruments or other purposes. The spacecraft appears to be surrounded by a dark blue-black background, representing the vastness of space, with numerous stars visible.

The overall impression is that of a spacecraft exploring the cosmos, capturing the essence of human curiosity and the desire to understand the universe.


## Using local images with data URLs
You can also use [data URLs](https://developer.mozilla.org/en-US/docs/Web/URI/Reference/Schemes/data) to send local images without uploading them to a remote server. This is useful for testing or when working with local files:

In [7]:
import base64
from pathlib import Path

# Read local image and convert to base64 data URL
def image_to_data_url(image_path):
    with open(image_path, "rb") as image_file:
        encoded_string = base64.b64encode(image_file.read()).decode('utf-8')
        file_extension = Path(image_path).suffix.lstrip('.')
        mime_type = f"image/{file_extension}" if file_extension in ['jpg', 'jpeg', 'png', 'gif'] else "image/jpeg"
        return f"data:{mime_type};base64,{encoded_string}"

# Convert local image to data URL
notebook_dir = os.path.dirname(os.path.abspath("multimodal-chat-x5.ipynb"))
image_path = os.path.join(notebook_dir, "media", "image.jpg")
data_url = image_to_data_url(image_path)

text_input = "What do you see in this image?"

response = client.chat.chat(
  model="palmyra-x5",
  messages=[
    {
      "role": "user",
      "content": [
        {
          "type": "text",
          "text": text_input
        },
        {
          "type": "image_url",
          "image_url": {
            "url": data_url
          }
        }
      ]
    }
  ]
)

print(response.choices[0].message.content)

The image depicts a spacecraft, specifically the Voyager 1, against a starry background. The spacecraft is positioned on the left side of the image, with its various components visible.

*   **Spacecraft Components:**
    *   A large white dish antenna is prominently displayed at the center of the spacecraft.
    *   Several booms and antennas are attached to the main body of the spacecraft.
    *   A gold-colored structure is visible at the top of the spacecraft, which appears to be a magnetometer or other scientific instrument.
*   **Background:**
    *   The background of the image is a deep blue-black color, representing space.
    *   Numerous stars are scattered throughout the background, giving the impression of being in deep space.
    *   A faint, blue-gray nebula or gas cloud is visible on the right side of the image, adding to the sense of depth and distance.

Overall, the image effectively conveys the sense of a spacecraft exploring the vastness of space, with its various c

## Multiple images with text
You can include multiple images in a single message:

In [9]:
text_input = "Compare these two images and tell me the differences:"
image_for_comparison = "https://hips.hearstapps.com/hmg-prod/images/o5i2rc7u3wbj4f2ybg6zyd-1657151459.jpg"

response = client.chat.chat(
  model="palmyra-x5",
  messages=[
    {
      "role": "user",
      "content": [
        {
          "type": "text",
          "text": text_input
        },
        {
          "type": "image_url",
          "image_url": {
            "url": image_input
          }
        },
        {
          "type": "image_url",
          "image_url": {
            "url": image_for_comparison
          }
        }
      ]
    }
  ]
)

print(response.choices[0].message.content)

The two images depict spacecraft designed for different purposes and environments.

**Image 1: Voyager Spacecraft**

*   The image on the left shows the Voyager spacecraft, a space probe designed to explore the outer Solar System and beyond.
*   It is equipped with a large dish antenna for communication with Earth.
*   The spacecraft is shown against a starry background, indicating its location in deep space.

**Image 2: Mars Rover**

*   The image on the right shows a Mars rover, a robotic vehicle designed to explore the surface of Mars.
*   The rover is equipped with various instruments, including cameras, spectrometers, and a robotic arm.
*   The rover is shown on a rocky, barren surface, indicating its location on Mars.

**Key Differences**

*   **Purpose**: The Voyager spacecraft is designed for interstellar exploration, while the Mars rover is designed for planetary exploration.
*   **Environment**: The Voyager spacecraft is in deep space, while the Mars rover is on the surface o